In [8]:
%run ../ts_boilerplate/dataprep_2.py

Model is not converging.  Current: 9681.170514175796 is not greater than 9681.173750027261. Delta is -0.0032358514654333703


Markdown feature documentation written to: ../docs/feature_documentation.md
(2891, 35)


In [9]:
final_df.head()

,PPH_Open,PPH_High,PPH_Low,PPH_Close,PPH_Volume,XPH_Open,XPH_High,XPH_Low,XPH_Close,XPH_Volume,...,XPH_Ratio_PPH_t-1,day_of_week,day_of_month,month,quarter,is_holiday_adjacent,days_to_cpi,days_since_cpi,days_to_nfp,days_since_nfp
Date,,,,,,,,,,,,,,,,,,,,,
2014-02-04,53.450001,53.700001,53.369999,53.680000,117000,44.595001,44.820000,44.215000,44.490002,180200,...,0.833804,1,4,2,1,0,9,24,32,2
2014-02-05,53.799999,53.959999,53.450001,53.840000,40400,44.430000,44.509998,43.535000,44.299999,184400,...,0.828800,2,5,2,1,0,8,25,31,3
2014-02-06,53.959999,54.090000,53.790001,53.959999,24200,44.610001,44.775002,43.970001,44.195000,76800,...,0.822808,3,6,2,1,0,7,26,30,4
2014-02-07,54.200001,54.959999,54.139999,54.959999,36100,44.419998,45.349998,44.244999,45.349998,186800,...,0.819033,4,7,2,1,0,6,27,29,5
2014-02-10,55.029999,55.349998,54.900002,55.349998,173000,45.424999,45.959999,45.185001,45.959999,127200,...,0.825146,0,10,2,1,0,3,30,26,8
